In [1]:

# In[1]:


# create one epoche, safe and evaluate manuel . after that, start again : try on GPU

import pykeen
from pykeen.pipeline import pipeline
import networkx as nx
import pathlib
from random import sample
import pandas as pd
from pykeen.triples import TriplesFactory
import torch
import numpy as np


# In[2]:


# from https://github.com/nebw/ehrgraphs/blob/master/ehrgraphs/data/data.py#L82-L117
def preprocess_graph_heterogeneous(graph: nx.Graph):
    edge_types = []
    for u, v, data in graph.edges.data():
        edge_types.append(data["edge_type"])

    edge_codes, edge_types = pd.factorize(edge_types)

    node_types = []
    for n, data in graph.nodes.data():
        node_types.append(data["node_type"])

    node_codes, node_types = pd.factorize(node_types)

    preprocessed_graph = nx.DiGraph()
    preprocessed_graph.add_nodes_from(graph.nodes())

    preprocessed_graph.node_codes = node_codes
    preprocessed_graph.node_types = node_types
        
    # drop shortcut edges
    exclude_codes = []
    exclude_codes.append(edge_codes[list(edge_types).index("Subsumes")])
    exclude_codes.append(edge_codes[list(edge_types).index("Is a")])

    for (u, v, w), c in zip(graph.edges.data("edge_weight"), edge_codes):
        assert w is not None

        # drop shortcut edges
        if c in exclude_codes and w < 1.0:
            continue

        preprocessed_graph.add_edge(u, v, edge_weight=w, edge_code=c)

    preprocessed_graph.edge_types = edge_types
    
    ## --- ##

    return preprocessed_graph


# In[ ]:


#loading the full graph
print('loading ...')
base_path = pathlib.Path(
    "/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/2_datasets_pre/211110_anewbeginning")
G = nx.readwrite.gpickle.read_gpickle('/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/2_datasets_pre/211110_anewbeginning/graph_full_211122.p')


# In[ ]:


# -----TEST -----#
print('preprocess ...')
SG = preprocess_graph_heterogeneous(G)


# In[ ]:


loading ...
preprocess ...


In [2]:
print('building array ... ')
tripleList=[]
nodes=[]
for u,v,data in SG.edges.data():
    l=[]
    l.append(u)
    nodes.append(u)
    l.append(data['edge_code'])
    l.append(v)
    nodes.append(v)
    tripleList.append(l)
print('length bevor sampling: ', len(tripleList))
#needs triples as ndarray - shape (n,3), dtype:str 
tripleArray=np.array(tripleList, dtype=str)


# In[ ]:



building array ... 
length bevor sampling:  28842781


In [ ]:

#2 Try out the defragmented version - first create triple factory
'''When continuing the training or in general using the model after resuming training, it is critical that the entity label to
identifier (entity_to_id) and relation label to identifier (relation_to_id) mappings are the same as the ones that were used 
when saving the checkpoint. If they are not, then any downstream usage will be nonsense. '''
tf = TriplesFactory.from_labeled_triples(tripleArray, create_inverse_triples=True)
print('TF done ...')

In [6]:


training_factory, testing_factory = tf.split([0.9 ,0.1])
entity_mapping = tf.entity_to_id
relation_mapping = tf.relation_to_id


#training = TriplesFactory.from_labeled_triples(training_factory, create_inverse_triples= True )

'''testing = TriplesFactory.from_labeled_triples(testing_factory, create_inverse_triples=True,
                                              entity_to_id=train.entity_to_id,
                                             relation_to_id=train.relation_to_id,)'''
                                             

# Pick a model
from pykeen.models import TransE

model = TransE(triples_factory=training_factory, embedding_dim=2)

# Pick an optimizer from Torch
from torch.optim import Adam
optimizer = Adam(params=model.get_grad_params())

# Pick a training approach !! contains the losses
from pykeen.training import SLCWATrainingLoop 
training_loop = SLCWATrainingLoop(

    model=model,

    
    triples_factory=training_factory,

    optimizer=optimizer,

)

# just run for one epoch, evaluate losses and restart training where it was left
for i in range(1,20):
    # Train like Cristiano Ronaldo
    losses = training_loop.train(

        triples_factory=training_factory,
        num_epochs=i,
        batch_size=256,
        checkpoint_name= 'gpu_run_own_small.pt',
        checkpoint_frequency=0

    )
    if i>1 and (losses[-2] - losses[-1]) < 0.0001:
        break



using automatically assigned random_state=3198982462
No cuda devices were available. The model runs on CPU
No random seed is specified. This may lead to non-reproducible results.


Training epochs on cpu:   0%|          | 0/1 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/202801 [00:00<?, ?batch/s]

KeyboardInterrupt: 